# Retrieval-Augmented Generation (RAG) Demonstration

Install required python packages

In [ ]:
!pip install datasets==2.19.1 langchain-community==0.2.0 langchain-chroma==0.1.1 sentence-transformers==2.7.0

Some imports we need to run the RAG demonstration.

In [ ]:
import chromadb

from typing import List, Optional

from datasets import load_dataset

from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_core.documents.base import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.vectorstores import VectorStore

Code to ignore warnings. Not a good code practice but fine for the demo.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

The generator should generate an answer based on the user query and the relevant documents.
We introduce a abstract class to work with generators and implementing a PromptGenerator.
The PromptGenerator is only creating a prompt which can be executed with any LLM you like.

In [ ]:
class Generator:
    def invoke(self, query: str, documents: List[Document]) -> str:
        pass

class PromptGenerator(Generator):
    def __init__(self):
        template = ("Use only the provided information following after \"Context:\" to answer the question following after \"Question:\" at the end.\n" +
                    "If you don't know the answer, just say that you don't know, don't try to make up an answer.\n" +
                    "Use three sentences maximum and keep the answer as concise as possible.\n\n" +
                    "Context: {context}\n\n" +
                    "Question: {question}")
        self.prompt_template = PromptTemplate.from_template(template)
        
    def invoke(self, query: str, documents: List[Document]) -> str:
        context = "\n".join([f"{i+1}. {doc.page_content}" for i, doc in enumerate(documents)])
        prompt = self.prompt_template.format(question=query, context=context)
        
        return prompt

We are not implementing the Retriever because there is already an implementation available in LangChain.
Instead, we will use that implementation, and we will wrap the creation of the retriever behind a Builder.
The Builder will also implement some logic to enhance the existing retriever with some new knowledge.

In [ ]:
class Builder:
    @staticmethod
    def create_retriever(docs: Optional[List[Document]]) -> VectorStore:
        collection_name="my_doc_store"
        embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        
        if docs:
            db = Chroma.from_documents(
                documents=docs,
                collection_name=collection_name,
                embedding=embedding_function,
            )
        else:
            db = Chroma(
                client=chromadb.Client(),
                collection_name=collection_name,
                embedding_function=embedding_function,
            )
        
        return db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.5})
    
    @staticmethod
    def add_knowledge(retriever: VectorStore, knowledge_collection: List[str]):
        retriever.add_documents([Document(page_content=knowledge) for knowledge in knowledge_collection])

To chain the retriever and generator together to implement our RAG we can make use of the Orchestrator.
Honestly, I don't know if there is a way to implement this with some LangChain pipeline, but I think for demonstration purpose that is enough. 

In [ ]:
class Orchestrator:
    def __init__(self, retriever: VectorStore, generator: Generator):
        self.retriever = retriever
        self.generator = generator
        
    def answer_question(self, question: str) -> str:
        relevant_documents = self.retriever.invoke(question)
        return self.generator.invoke(question, relevant_documents)

Now that we have all of our components let's initialize our RAG with some data about the seven wonders

In [ ]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(page_content=doc["content"], meta=doc["meta"]) for doc in dataset]

retriever = Builder.create_retriever(docs)
generator = PromptGenerator()

rag = Orchestrator(retriever, generator)

Let's see what our RAG is returning

In [ ]:
print(rag.answer_question("What happened to the Tomb of Mausolus?"))

Let's try what will happen when we ask our RAG about some stuff around our sun.

In [ ]:
print(rag.answer_question("How hot is the sun?"))

Obviously our RAG can't give us any answer about the sun because it only includes some knowledge about the seven wonders.
That's a little bit sad when we want to know something about the sun but also very great because we don't see any hallucinated answer.
But we still want to get some answers about the sun so let's add some new knowledge!

In [ ]:
Builder.add_knowledge(retriever, ["The sun is a star located at the center of our solar system.", "The sun is extremely hot!", "The sun has a very high temperature."])
print(rag.answer_question("How hot is the sun?"))

Nice now we are able to find the new knowledge that the sun is really hot.
To finish our demonstration we can also show you how important document chunking is and what impact it can have to get the knowledge we want.
We will add some information about the sun's core temperature, but we will add some additional information which is not related to the temperature of the sun.

In [ ]:
Builder.add_knowledge(retriever, ["Here is some text to hide some interesting and useful information. The cores temperature of the sun core is approximately 15 million degrees Celsius. That should demonstrate why and how important document chunking is."])
print(rag.answer_question("How hot is the sun?"))

We will not find the information about the temperature of the sun even when it is available in our knowledge base.